<a href="https://colab.research.google.com/github/alsindirose-afk/Dal-Shemagh-Challenge-Solution/blob/main/Copy_of_RENEWED_Dal_Shemagh_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
from sklearn.model_selection import train_test_split
import shutil
import random
from PIL import Image

In [ ]:
# @title
# Creating a function to calculate IOU for different boxes

def calculate_iou(box1, box2):
    """
    box1, box2: [x, y, w, h] в относительных координатах (0..1)
    """
    b1_x1, b1_x2 = box1[0] - box1[2]/2, box1[0] + box1[2]/2
    b1_y1, b1_y2 = box1[1] - box1[3]/2, box1[1] + box1[3]/2
    b2_x1, b2_x2 = box2[0] - box2[2]/2, box2[0] + box2[2]/2
    b2_y1, b2_y2 = box2[1] - box2[3]/2, box2[1] + box2[3]/2

    inter_x1 = max(b1_x1, b2_x1)
    inter_y1 = max(b1_y1, b2_y1)
    inter_x2 = min(b1_x2, b2_x2)
    inter_y2 = min(b1_y2, b2_y2)

    inter_area = max(0, inter_x2 - inter_x1) * max(0, inter_y2 - inter_y1)
    b1_area = (b1_x2 - b1_x1) * (b1_y2 - b1_y1)
    b2_area = (b2_x2 - b2_x1) * (b2_y2 - b2_y1)
    union_area = b1_area + b2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

In [ ]:
# @title
# a function to calculate the distance between the centers of 2 boxes
def calculate_distance(box1, box2):
  x1, y1 = box1[0], box1[1]
  x2, y2 = box2[0], box2[1]

  return np.sqrt((x2-x1)**2+(y2-y1)**2)

In [ ]:
# @title
# creating a csv file for the data extracted from the images' label files

images_path = "/content/drive/MyDrive/Datasets/images_dal_shemagh/train"
labels_path = "/content/drive/MyDrive/Datasets/labels_dal_shemagh/train"

images = glob.glob(os.path.join(images_path, "*.jpg"))

metadata_path = "/content/drive/MyDrive/Datasets/Shemaq Challenge Dal/train_labels.csv"
csv_file = pd.read_csv(metadata_path)

X  = []
labels = []

for i, img_path in enumerate(images):

  filename = img_path.split("/")[-1]

  label = os.path.join(labels_path, filename.split(".")[0]+".txt")
  f = open(label, "r")

  boxes = f.readlines()
  f.close()

  right_place = csv_file['right_place'].iloc[i]


  person_boxes = []
  shemagh_boxes = []
  diagonal_slopes = []

  diagonal_slope = None



  if len(boxes) == 0:
    continue

  else:
    for box in boxes:
        box = box.split(" ")
        cls = float(box[0])
        x, y, w, h = float(box[1]), float(box[2]), float(box[3]), float(box[4])
        diagonal_slopes.append(h/w)



        if cls == 0:
            person_boxes.append([x, y, w, h])

        elif cls == 1:
            shemagh_boxes.append([x, y, w, h])


    best_iou = 0.0
    min_distance = 640
    best_pair = None

    if person_boxes and shemagh_boxes:

      for p_idx, p_box in enumerate(person_boxes):
          for s_idx, s_box in enumerate(shemagh_boxes):
            iou = calculate_iou(p_box, s_box)
            dist = calculate_distance(p_box, s_box)
            if iou > best_iou:
                best_iou = iou
                best_pair = (p_idx, s_idx)
            min_distance = min(dist, min_distance)

      if best_pair is not None:
          p_box = person_boxes[best_pair[0]]
          s_box = shemagh_boxes[best_pair[1]]
          iou_threshold = 0.01
          vertical_threshold = 0.0
          min_area_ratio = 0.4

          iou_ok = best_iou > iou_threshold
          vertical_ok = s_box[1] < p_box[1] - vertical_threshold
          area_ratio = (s_box[2] * s_box[3]) / (p_box[2] * p_box[3])
          size_ok = area_ratio > min_area_ratio

          if iou_ok and size_ok:
            h1 = person_boxes[best_pair[0]][3]
            h2 = shemagh_boxes[best_pair[1]][3]
            w1 = person_boxes[best_pair[0]][2]
            w2 = shemagh_boxes[best_pair[1]][2]
            diagonal_slope = max(h1, h2)/max(w1, w2)


  if diagonal_slope is None:
    diagonal_slope = np.mean(diagonal_slopes)

  features = []
  features.append(filename)
  features.append(best_iou)
  features.append(min_distance)
  features.append(diagonal_slope)
  X.append(features)
  labels.append(float(right_place))



cls_df = pd.DataFrame(X, columns=['img_name', 'max_iou', 'min_distance', 'mean_box_diagonal_slope'])
cls_df['label'] = labels

In [ ]:
cls_df.to_csv("binary_classification_renewed.csv", index=False)

max_iou = max(cls_df['max_iou'])
print(max_iou)

img_name = cls_df[cls_df['max_iou']==max_iou]['img_name']
print(img_name)

0.41146939978675706
241    393.jpg
Name: img_name, dtype: object


In [ ]:
path_to_data_yaml = '/content/drive/MyDrive/Datasets/Shemaq Challenge Dal/data.yaml'

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.3 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
model = YOLO("/content/yolo8n_baseline_with_augmentations0.pt")

In [ ]:
path_to_color_augmented_images = "/content/drive/MyDrive/Datasets/Color_Augmented_Positive_Images"

**Getting Results For The Augmented Images to Include Them in The CSV File to Train The Classification CNN on a Balanced Dataset**

In [ ]:
# @title

augmented_images = glob.glob(os.path.join(path_to_color_augmented_images, "*.jpg"))

X = []
labels = []

for image in augmented_images:

  img_name = image.split("/")[-1]



  right_place = 1


  results = model.predict(image, conf=0.25, verbose=False)[0]

  person_boxes = []
  shemagh_boxes = []
  diagonal_slopes = []

  diagonal_slope = None

  if len(results.boxes) == 0:
    continue

  else:
    for box in results.boxes:
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        x, y, w, h = box.xywhn[0].tolist()

        diagonal_slopes.append(h/w)



        if cls == 0:
            person_boxes.append([x, y, w, h])
        elif cls == 1:
            shemagh_boxes.append([x, y, w, h])


    best_iou = 0.0
    best_pair = None
    min_distance = 640

    if person_boxes and shemagh_boxes:

          for p_idx, p_box in enumerate(person_boxes):

              for s_idx, s_box in enumerate(shemagh_boxes):

                  iou = calculate_iou(p_box, s_box)
                  dist = calculate_distance(p_box, s_box)

                  if iou > best_iou:
                      best_iou = iou
                      best_pair = (p_idx, s_idx)

                  min_distance = min(dist, min_distance)

          if best_pair is not None:
              p_box = person_boxes[best_pair[0]]
              s_box = shemagh_boxes[best_pair[1]]

    else:
      continue
#-------------------------------------------------------------------------
  if diagonal_slope is None:
    diagonal_slope = np.mean(diagonal_slopes)

  features = []
  features.append(filename)
  features.append(best_iou)
  features.append(min_distance)
  features.append(diagonal_slope)
  X.append(features)
  labels.append(float(right_place))

In [ ]:
print(len(X), len(labels))

362 362


In [ ]:
cls_df = pd.DataFrame(X, columns=['img_name', 'max_iou', 'min_distance', 'mean_box_diagonal_slope'])
cls_df['label'] = labels

In [ ]:
cls_df.to_csv("binary_classification_color_aug_renewed.csv", index=False)

max_iou = max(cls_df['max_iou'])
print(max_iou)

img_name = cls_df[cls_df['max_iou']==max_iou]['img_name']
print(img_name)

0.6347596840063772
190    82.jpg
Name: img_name, dtype: object


**Creating The Training & Validation Loop Functions**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import f1_score

In [ ]:
def train_one_epoch(model, train_loader, optimizer, criterion, threshold, device):
  model.train()
  total_loss = 0
  total = 0
  total_f1 = 0

  for X, labels in tqdm(train_loader):
    X, labels = X.to(device), labels.view(-1, 1).to(device)

    outputs = model(X)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    pred = (outputs>=threshold)

    total_loss += loss.item()
    total += labels.size(0)
    total_f1 += f1_score(labels.data, pred, zero_division=0)

  avg_loss = total_loss/len(train_loader)
  f1 = total_f1/len(train_loader)

  return avg_loss, f1

In [ ]:
def validate(model, train_loader, criterion, threshold, device):
  model.eval()
  total_loss = 0
  total = 0
  total_f1 = 0

  with torch.no_grad():
    for X, labels in tqdm(train_loader):
      X, labels = X.to(device), labels.view(-1, 1).to(device)

      outputs = model(X)
      loss = criterion(outputs, labels)


      pred = (outputs>=threshold)

      total_loss += loss.item()
      total += labels.size(0)
      total_f1 += f1_score(labels.data, pred, zero_division=0)

  avg_loss = total_loss/len(train_loader)
  f1 = total_f1/len(train_loader)

  return avg_loss, f1

**Building The Classification Neural Network**

In [ ]:
class BinaryClassificationNN(nn.Module):
  def __init__(self, in_features, hidden_features, out_features, dropout_rate1, dropout_rate2):
    super(BinaryClassificationNN, self).__init__()

    self.network = nn.Sequential(
        nn.Linear(in_features, hidden_features),
        nn.BatchNorm1d(hidden_features),
        nn.ReLU(inplace=True),
        nn.Dropout(dropout_rate1),

        nn.Linear(hidden_features, hidden_features),
        nn.BatchNorm1d(hidden_features),
        nn.ReLU(inplace=True),
        nn.Dropout(dropout_rate2),

        nn.Linear(hidden_features, hidden_features),
        nn.BatchNorm1d(hidden_features),
        nn.ReLU(inplace=True),

        nn.Linear(hidden_features, out_features),
        nn.BatchNorm1d(out_features),
        nn.Sigmoid()
    )

  def forward(self, x):
    return self.network(x)

**Applying The Classification Model With Equal Data For Both Classes On Color Augmented Images**

In [ ]:
original_data = pd.read_csv("/content/binary_classification_renewed.csv")
new_data= pd.read_csv("/content/binary_classification_color_aug_renewed.csv")
#new_data = new_data.iloc[:512,]
all_color_data = pd.concat([original_data, new_data], axis=0)

#Removing Duplicates
all_color_data.drop_duplicates(inplace=True)
all_color_data.to_csv("Final_Metadata_Renewed.csv", index=False)
all_color_data.shape

(808, 5)

In [ ]:
# Now that the data is balanced, we can use the same dropout rate and make it low

metadata = pd.read_csv("/content/Final_Metadata_Renewed.csv")

X = metadata.drop(columns=['img_name'
, 'label'])
y = metadata['label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.2, random_state=42, shuffle=True, stratify=y)

X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)

X_val = torch.tensor(X_val.values, dtype=torch.float32)
y_val = torch.tensor(y_val.values, dtype=torch.float32)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

BATCH_SIZE=32

train_loader = DataLoader(train_dataset,  batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
classification_model = BinaryClassificationNN(in_features=3, hidden_features =32, out_features=1, dropout_rate1=.4, dropout_rate2=.4)


THRESHOLD = .5
LEARNING_RATE=.003

optimizer = optim.Adam(classification_model.parameters(), lr=LEARNING_RATE)
criterion = nn.BCELoss()

NUM_EPOCHS = 20

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_losses = []
val_losses = []

train_f1s = []
val_f1s = []

for epoch in range(NUM_EPOCHS):

  train_loss, train_f1 = train_one_epoch(classification_model, train_loader, optimizer, criterion, THRESHOLD, device)
  val_loss, val_f1 = validate(classification_model, val_loader, criterion, THRESHOLD,  device)

  train_losses.append(train_loss)
  val_losses.append(val_losses)
  train_f1s.append(train_f1)
  val_f1s.append(val_f1)

  print(f"Epoch[{epoch+1}/{NUM_EPOCHS}] | Training Loss: {train_loss} | Validation Loss: {val_loss} | Training F1 Score: {train_f1} | Validation F1 Score: {val_f1}")

100%|██████████| 6/6 [00:00<00:00, 221.43it/s]


Epoch[1/20] | Training Loss: 0.4593489780312493 | Validation Loss: 0.3604273994763692 | Training F1 Score: 0.868540247217455 | Validation F1 Score: 0.9395036837913183


100%|██████████| 6/6 [00:00<00:00, 151.18it/s]


Epoch[2/20] | Training Loss: 0.4046177253836677 | Validation Loss: 0.3722159266471863 | Training F1 Score: 0.9061057099608447 | Validation F1 Score: 0.9395036837913183


100%|██████████| 6/6 [00:00<00:00, 151.06it/s]


...

Epoch[18/20] | Training Loss: 0.2678814750342142 | Validation Loss: 0.25369218985239667 | Training F1 Score: 0.9264110576484113 | Validation F1 Score: 0.9395036837913183


100%|██████████| 6/6 [00:00<00:00, 206.88it/s]


Epoch[19/20] | Training Loss: 0.2682367726450875 | Validation Loss: 0.24646454056104025 | Training F1 Score: 0.9268698022928215 | Validation F1 Score: 0.9395036837913183


100%|██████████| 6/6 [00:00<00:00, 235.17it/s]

Epoch[20/20] | Training Loss: 0.2882150816065924 | Validation Loss: 0.24308781325817108 | Training F1 Score: 0.9138879307813783 | Validation F1 Score: 0.9395036837913183


In [ ]:
torch.save(classification_model.state_dict(), "classification_model_renewed.pt")

**Importing The Models Trained in The Previous Notebook**

In [ ]:
model9t = YOLO("/content/yolo9t_finetuned.pt")


In [ ]:
model11n = YOLO("yolov11n_finetuned.pt")

In [ ]:
model8n = YOLO("/content/yolo8n_baseline_with_augmentations0.pt")

In [ ]:
classification_model = classification_model = BinaryClassificationNN(in_features=3, hidden_features =32, out_features=1, dropout_rate1=.4, dropout_rate2=.4)

state_dict = torch.load("/content/classification_model_renewed.pt")

classification_model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
# @title
# Running the whole pipeline on the test images

test_images = glob.glob(os.path.join("/content/drive/MyDrive/Datasets/Shemaq Challenge Dal", "images", "test", "*.jpg"))
submission_data = []


for i, img_path in enumerate(test_images):


  filename = img_path.split("/")[-1]


  results = model.predict(img_path, conf=0.25, verbose=False)[0]

  prediction_strings = []
  person_boxes = []
  shemagh_boxes = []
  diagonal_slopes = []

  diagonal_slope = None

  if len(results.boxes) == 0:
    prediction_str = "-"
    right_place = 0

  else:
    for box in results.boxes:
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        x, y, w, h = box.xywhn[0].tolist()

        diagonal_slopes.append(h/w)

        prediction_strings.append(f"{cls} {conf:.4f} {x:.4f} {y:.4f} {w:.4f} {h:.4f}")

        if cls == 0:
            person_boxes.append([x, y, w, h])
        elif cls == 1:
            shemagh_boxes.append([x, y, w, h])

    prediction_str = " ".join(prediction_strings)


    #right_place = 0
    best_iou = 0.0
    best_pair = None
    min_distance = 640

    if person_boxes and shemagh_boxes:

      for p_idx, p_box in enumerate(person_boxes):
          for s_idx, s_box in enumerate(shemagh_boxes):
                iou = calculate_iou(p_box, s_box)
                dist = calculate_distance(p_box, s_box)
                if iou > best_iou:
                    best_iou = iou
                    best_pair = (p_idx, s_idx)
                min_distance = min(dist, min_distance)

      if best_pair is not None:
          p_box = person_boxes[best_pair[0]]
          s_box = shemagh_boxes[best_pair[1]]
          iou_threshold = 0.01
          vertical_threshold = 0.0
          min_area_ratio = 0.4

          iou_ok = best_iou > iou_threshold
          vertical_ok = s_box[1] < p_box[1] - vertical_threshold
          area_ratio = (s_box[2] * s_box[3]) / (p_box[2] * p_box[3])
          size_ok = area_ratio > min_area_ratio

          if iou_ok and size_ok:
            h1 = person_boxes[best_pair[0]][3]
            h2 = shemagh_boxes[best_pair[1]][3]
            w1 = person_boxes[best_pair[0]][2]
            w2 = shemagh_boxes[best_pair[1]][2]
            diagonal_slope = max(h1, h2)/max(w1, w2)

      if diagonal_slope is None:
        diagonal_slope = np.mean(diagonal_slopes)

      vec = np.array([max_iou, min_distance, diagonal_slope], dtype=np.float32)
      vec = torch.from_numpy(vec)

      classification_model.eval()
      with torch.no_grad():
        right_place = float(classification_model(vec.unsqueeze(0)).cpu())
      right_place = int(right_place>.5)


    else:
      right_place = 0



#--------------------------------------------------------------------------

  submission_data.append([filename, right_place, prediction_str])

df_sub = pd.DataFrame(submission_data, columns=['filename', 'right_place', 'prediction_string'])
df_sub.to_csv('submission_8n_renewed.csv', index=False)


In [ ]:
df_sub[df_sub['right_place']==1].shape

(53, 3)

In [ ]:
df_sub.shape

(842, 3)

**Creating The Ensembled Model Using The Optimal Weights Found in The Previous Notebook**

In [ ]:
model9t = YOLO("/content/yolo9t_finetuned.pt")
model11n = YOLO("/content/yolov11n_finetuned.pt")
model8n = YOLO("/content/yolo8n_baseline_with_augmentations0.pt")
models = [model9t, model11n, model8n]

weights_file = open("/content/drive/MyDrive/Datasets/Shemaq Challenge Dal/ensemble_learning_weights.txt", "r")
theta = np.array((weights_file.readlines()[-1].split(",")), dtype=np.float32)

print(theta)

[    0.37996      0.3866     0.30274]


In [ ]:
def yolo_ensembled(models, theta, image):

  N = len(models)

  img_name = image.split("/")[-1]

  all_models_vectors = []

  for j, model in enumerate(models):

    results = model.predict(image, conf=.25, verbose=False)[0]

    model_vec = np.zeros((len(results.boxes), 6))

    for i, box in enumerate(results.boxes):


      conf = float(box.conf[0])
      cls = int(box.cls[0])
      x, y, w, h = box.xywhn[0].tolist()

      #cls_pred = conf if cls == 1 else 1-conf
      pred_vec = np.array([cls, conf, x, y, w, h])
      model_vec[i, :] = pred_vec

    all_models_vectors.append(model_vec*theta[j])

  max_num_boxes = max(vec.shape[0] for vec in all_models_vectors) # finding the maximum number of boxes found in the image
  boxes = []

  for n in range(max_num_boxes):

    box = np.zeros(6)
    for j, model in enumerate(models):

      model_vector = all_models_vectors[j]

      if model_vector.shape[0]>n:
        box += theta[j]*model_vector[n, :]
        box[0] = max(box[0], model_vector[n, 0])
      else:
        box = []
        break

    if len(box)>0:
      box[0] = float(box[0]>=.5)
      boxes.append(box)


  return boxes

**Getting Results For The Augmented Images Using The Ensembling Model to Include Them in The CSV File to Train The Classification CNN on a Balanced Dataset**

In [ ]:
# @title

augmented_images = glob.glob(os.path.join(path_to_color_augmented_images, "*.jpg"))

X = []
labels = []

for image in augmented_images:

  filename = image.split("/")[-1]



  right_place = 1


  boxes = yolo_ensembled(models, theta, image)

  person_boxes = []
  shemagh_boxes = []
  diagonal_slopes = []

  diagonal_slope = None

  if len(boxes) == 0:
    continue

  else:
    for box in boxes:
        cls = int(box[0])
        conf = float(box[1])
        print(f"conf{conf},cls{cls}")

        if conf<0.15:
          continue



        x, y, w, h = box[2], box[3], box[4], box[5]

        diagonal_slopes.append(h/w)



        if cls == 0:
            person_boxes.append([x, y, w, h])
        elif cls == 1:
            shemagh_boxes.append([x, y, w, h])


    best_iou = 0.0
    best_pair = None
    min_distance = 640

    if person_boxes and shemagh_boxes:

          for p_idx, p_box in enumerate(person_boxes):

              for s_idx, s_box in enumerate(shemagh_boxes):

                  iou = calculate_iou(p_box, s_box)
                  dist = calculate_distance(p_box, s_box)

                  if iou > best_iou:
                      best_iou = iou
                      best_pair = (p_idx, s_idx)

                  min_distance = min(dist, min_distance)

          if best_pair is not None:
              p_box = person_boxes[best_pair[0]]
              s_box = shemagh_boxes[best_pair[1]]

    else:
      continue
#-------------------------------------------------------------------------
  if diagonal_slope is None:
    diagonal_slope = np.mean(diagonal_slopes)

  features = []
  features.append(filename)
  features.append(best_iou)
  features.append(min_distance)
  features.append(diagonal_slope)
  X.append(features)
  labels.append(float(right_place))

conf0.24988721554471494,cls0
conf0.16418649009959146,cls1
conf0.19093236132556332,cls0
conf0.1810341855078923,cls0
conf0.24854438723049835,cls0
conf0.15700914586465842,cls1
conf0.25482216800940205,cls0
conf0.15787252584824799,cls1
conf0.24739962097883475,cls0
conf0.15994141888170274,cls1
conf0.2561710643261693,cls0
conf0.18156977692800638,cls1
conf0.24464857597871803,cls0
conf0.16108050215247438,cls1
conf0.2596163601852546,cls0
conf0.16099022742262786,cls1
conf0.2501033687254891,cls0
conf0.16217760011587304,cls1
conf0.2384278207753578,cls0
conf0.2280586747128594,cls0
conf0.23667985607293518,cls0
conf0.16154199045315837,cls1
conf0.2525815574545016,cls0
conf0.1816954754591516,cls1
conf0.2551617753620839,cls0
conf0.24100547350297546,cls0
conf0.166621887881165,cls1
conf0.23152819247106354,cls0
conf0.2231156000614447,cls0
conf0.1741493518335995,cls1
conf0.23320014560266425,cls0
conf0.1639043632907624,cls1
conf0.2541300663452899,cls0
conf0.16870285200500756,cls1
conf0.2494765119453722,cls0
c

In [ ]:
print(len(X), len(labels))

137 137


In [ ]:
cls_df = pd.DataFrame(X, columns=['img_name', 'max_iou', 'min_distance', 'mean_box_diagonal_slope'])
cls_df['label'] = labels

In [ ]:
cls_df.to_csv("binary_classification_color_aug_renewed_ensembled.csv", index=False)

max_iou = max(cls_df['max_iou'])
print(max_iou)

img_name = cls_df[cls_df['max_iou']==max_iou]['img_name']
print(img_name)

0.8288856987061657
65    715.jpg
Name: img_name, dtype: object


**Applying The Classification Model With Equal Data For Both Classes On Color Augmented Images**

In [ ]:
original_data = pd.read_csv("/content/binary_classification_renewed.csv")
new_data= pd.read_csv("/content/binary_classification_color_aug_renewed_ensembled.csv")
#new_data = new_data.iloc[:512,]
all_color_data = pd.concat([original_data, new_data], axis=0)

#Removing Duplicates
all_color_data.drop_duplicates(inplace=True)
all_color_data.to_csv("Final_Metadata_Renewed_Ensembled.csv", index=False)
all_color_data.shape

(813, 5)

In [ ]:
# Now that the data is balanced, we can use the same dropout rate and make it low

metadata = pd.read_csv("/content/Final_Metadata_Renewed_Ensembled.csv")

X = metadata.drop(columns=['img_name'
, 'label'])
y = metadata['label']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=.2, random_state=42, shuffle=True, stratify=y)

X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32)

X_val = torch.tensor(X_val.values, dtype=torch.float32)
y_val = torch.tensor(y_val.values, dtype=torch.float32)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

BATCH_SIZE=32

train_loader = DataLoader(train_dataset,  batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
classification_model = BinaryClassificationNN(in_features=3, hidden_features =32, out_features=1, dropout_rate1=.2, dropout_rate2=.2)


THRESHOLD = .5
LEARNING_RATE=.01

optimizer = optim.Adam(classification_model.parameters(), lr=LEARNING_RATE)
criterion = nn.BCELoss()

NUM_EPOCHS = 20

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
train_losses = []
val_losses = []

train_f1s = []
val_f1s = []

for epoch in range(NUM_EPOCHS):

  train_loss, train_f1 = train_one_epoch(classification_model, train_loader, optimizer, criterion, THRESHOLD, device)
  val_loss, val_f1 = validate(classification_model, val_loader, criterion, THRESHOLD,  device)

  train_losses.append(train_loss)
  val_losses.append(val_losses)
  train_f1s.append(train_f1)
  val_f1s.append(val_f1)

  print(f"Epoch[{epoch+1}/{NUM_EPOCHS}] | Training Loss: {train_loss} | Validation Loss: {val_loss} | Training F1 Score: {train_f1} | Validation F1 Score: {val_f1}")

100%|██████████| 6/6 [00:00<00:00, 155.03it/s]


Epoch[1/20] | Training Loss: 0.4576054783094497 | Validation Loss: 0.305479201177756 | Training F1 Score: 0.8535656732829932 | Validation F1 Score: 0.9374762157115099


100%|██████████| 6/6 [00:00<00:00, 154.97it/s]


Epoch[2/20] | Training Loss: 0.362669533207303 | Validation Loss: 0.32519830763339996 | Training F1 Score: 0.9250814245190194 | Validation F1 Score: 0.9374762157115099


100%|██████████| 6/6 [00:00<00:00, 178.36it/s]


...

Epoch[18/20] | Training Loss: 0.23614913757358277 | Validation Loss: 0.20571739599108696 | Training F1 Score: 0.9250321339195886 | Validation F1 Score: 0.9374762157115099


100%|██████████| 6/6 [00:00<00:00, 236.41it/s]


Epoch[19/20] | Training Loss: 0.24078977711143948 | Validation Loss: 0.20490210627516112 | Training F1 Score: 0.9242302179827345 | Validation F1 Score: 0.9374762157115099


100%|██████████| 6/6 [00:00<00:00, 257.47it/s]

Epoch[20/20] | Training Loss: 0.23654744986976897 | Validation Loss: 0.2071282962958018 | Training F1 Score: 0.9253231504964913 | Validation F1 Score: 0.9374762157115099


In [ ]:
torch.save(classification_model.state_dict(), "classification_model_renewed_ensembled.pt")

**Running The Test Images Through The Final Ensembling Pipeline**

In [ ]:
classification_model = classification_model = BinaryClassificationNN(in_features=3, hidden_features =32, out_features=1, dropout_rate1=.2, dropout_rate2=.2)

state_dict = torch.load("/content/classification_model_renewed_ensembled.pt")

classification_model.load_state_dict(state_dict)

<All keys matched successfully>

In [ ]:
# Running the test images through the ensembled pipeline
test_images = glob.glob(os.path.join("/content/drive/MyDrive/Datasets/Shemaq Challenge Dal", "images", "test", "*.jpg"))
submission_data = []


for i, img_path in enumerate(test_images):


  filename = img_path.split("/")[-1]


  boxes = yolo_ensembled(models, theta, img_path)

  prediction_strings = []
  person_boxes = []
  shemagh_boxes = []
  diagonal_slopes = []

  diagonal_slope = None

  if len(boxes) == 0:
    prediction_str = "-"
    right_place = 0

  else:
    for box in boxes:
        cls = box[0]
        conf = float(box[1])

        if conf<0.15:
          continue

        x, y, w, h = box[2], box[3], box[4], box[5]

        diagonal_slopes.append(h/w)

        prediction_strings.append(f"{cls} {conf:.4f} {x:.4f} {y:.4f} {w:.4f} {h:.4f}")

        if cls == 0:
            person_boxes.append([x, y, w, h])
        elif cls == 1:
            shemagh_boxes.append([x, y, w, h])

    prediction_str = " ".join(prediction_strings)


    #right_place = 0
    best_iou = 0.0
    best_pair = None
    min_distance = 640

    if person_boxes and shemagh_boxes:

      for p_idx, p_box in enumerate(person_boxes):
          for s_idx, s_box in enumerate(shemagh_boxes):
                iou = calculate_iou(p_box, s_box)
                dist = calculate_distance(p_box, s_box)
                if iou > best_iou:
                    best_iou = iou
                    best_pair = (p_idx, s_idx)
                min_distance = min(dist, min_distance)

      if best_pair is not None:
          p_box = person_boxes[best_pair[0]]
          s_box = shemagh_boxes[best_pair[1]]
          iou_threshold = 0.01
          vertical_threshold = 0.0
          min_area_ratio = 0.4

          iou_ok = best_iou > iou_threshold
          vertical_ok = s_box[1] < p_box[1] - vertical_threshold
          area_ratio = (s_box[2] * s_box[3]) / (p_box[2] * p_box[3])
          size_ok = area_ratio > min_area_ratio

          if iou_ok and size_ok:
            h1 = person_boxes[best_pair[0]][3]
            h2 = shemagh_boxes[best_pair[1]][3]
            w1 = person_boxes[best_pair[0]][2]
            w2 = shemagh_boxes[best_pair[1]][2]
            diagonal_slope = max(h1, h2)/max(w1, w2)

      if diagonal_slope is None:
        diagonal_slope = np.mean(diagonal_slopes)

      vec = np.array([best_iou, min_distance, diagonal_slope], dtype=np.float32)
      vec = torch.from_numpy(vec)

      classification_model.eval()
      with torch.no_grad():
        right_place = float(classification_model(vec.unsqueeze(0)).cpu())
      right_place = int(right_place>.5)


    else:
      right_place = 0



#--------------------------------------------------------------------------

  submission_data.append([filename, right_place, prediction_str])

df_sub = pd.DataFrame(submission_data, columns=['filename', 'right_place', 'prediction_string'])
df_sub.to_csv('submission_renewed_ensembled_3.csv', index=False)


In [ ]:
df_sub = pd.DataFrame(submission_data, columns=['filename', 'right_place', 'prediction_string'])
df_sub.to_csv('submission_ensembled.csv', index=False)

In [ ]:
df_sub[df_sub['right_place']==1].shape

(29, 3)

In [ ]:
df_sub.shape

(842, 3)